In [1]:
import dbmanager

In [2]:
db = dbmanager.DBManager()
db.init_db()

In [3]:
db.query_users()

[]

In [4]:
from faker import Faker
from collections import defaultdict
fake = Faker()

In [5]:
from datetime import datetime 

start_time = datetime.now() 

fake_data = defaultdict(list)
for _ in range(1000000):
    fake_data["first_name"].append( fake.first_name() )
    fake_data["last_name"].append( fake.last_name() )
    
time_elapsed = datetime.now() - start_time

print('time elapsed {}'.format(time_elapsed))

time elapsed 0:02:58.744524


In [7]:
start_time = datetime.now() 
for ind in range(len(fake_data["first_name"])):
    db.add_user_short(fake_data["first_name"][ind], fake_data["last_name"][ind])
    
    if ind%1000 == 0:
        time_elapsed = datetime.now() - start_time
        print(str(ind) + ', time elapsed {}'.format(time_elapsed))

0, time elapsed 0:00:00.009669
1000, time elapsed 0:00:05.972008
2000, time elapsed 0:00:12.155172
3000, time elapsed 0:00:19.231738
4000, time elapsed 0:00:43.173794
5000, time elapsed 0:01:08.798009
6000, time elapsed 0:01:33.809006
7000, time elapsed 0:01:58.343007
13000, time elapsed 0:04:20.951514
14000, time elapsed 0:04:44.254264
15000, time elapsed 0:05:07.293599
16000, time elapsed 0:05:30.305072
17000, time elapsed 0:05:53.152334
18000, time elapsed 0:06:16.029952
19000, time elapsed 0:06:38.732471
20000, time elapsed 0:07:01.352543
21000, time elapsed 0:07:24.073674
22000, time elapsed 0:07:46.732817
23000, time elapsed 0:08:09.178332
24000, time elapsed 0:08:31.432368
25000, time elapsed 0:08:53.733218
26000, time elapsed 0:09:15.864696
27000, time elapsed 0:09:37.856839
28000, time elapsed 0:09:59.947017
29000, time elapsed 0:10:22.209799
30000, time elapsed 0:10:44.321741
31000, time elapsed 0:11:06.395964
32000, time elapsed 0:11:28.490843
33000, time elapsed 0:11:50.642